# Instalación y carga de datos desde la API

In [ ]:
!pip install sodapy


In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.datos.gov.co", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(www.datos.gov.co,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ch4u-f3i5", limit=100000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
# Dimensiones de la tabla descargada
results_df.shape

In [ ]:
# Carga del archivo CSV
# desde el directorio de /kaggle/input/

import pandas as pd

df = pd.read_csv("/kaggle/input/suelos/Resultados_de_Anlisis_de_Laboratorio_Suelos_en_Colombia_20251124.csv")
df.head()


In [ ]:
#Muestra cuántas filas y columnas tiene tu DataFrame df.
df.shape

In [ ]:
#Lista todos los nombres de columnas del DataFrame
df.columns

In [ ]:
# Filtra todas las filas que tienen al menos un valor nulo (NaN) en alguna columna.
df[df.isnull().any(axis=1)]

In [ ]:
# Cuenta cuántos valores nulos hay en cada columna.
df.isnull().sum()

In [ ]:
#Muestra el tipo de dato de cada columna
df.dtypes

In [ ]:
#Resume la estructura del DataFrame:

#número de filas
#número de columnas
#tipo de cada columna
#cuántos valores no nulos hay por columna.

df.info()

In [ ]:
#Dice cuántos valores distintos tiene cada columna (sirve para ver si es categórica, numérica con pocas categorías, etc.).
df.nunique()

In [ ]:
#Recorre todas las columnas.

#Para cada una:
#imprime el nombre de la columna,
#imprime todos los valores únicos que hay en esa columna.
#Útil para revisar categorías o ver si hay valores raros como "ND", "NR", etc.

for col in df.columns:
    print(f"\nColumna: {col}")
    print(df[col].unique())


In [ ]:
#Similar a la anterior, pero ahora imprime el conteo de cada valor distinto (frecuencias).

#Sirve para ver:
#qué valores son más frecuentes,
#si hay valores “basura” que aparecen pocas veces,
#distribución de categorías.

for col in df.columns:
    print(f"\n=== {col} ===")
    print(df[col].value_counts())


In [ ]:
#Crea un DataFrame summary donde:

#"grupos_totales" = número de valores únicos por columna.
#"grupos" = el listado de valores únicos de cada columna.
#Guarda ese resumen en un CSV grupos_por_columna.csv.
#Muestra summary en pantalla.


summary = pd.DataFrame({
    "grupos_totales": df.nunique(),
    "grupos": df.apply(lambda x: x.unique())
})

summary.to_csv("grupos_por_columna.csv", index=True)

summary


In [ ]:
#Filtra filas donde la columna "Zinc disponible doble acido" (nota el espacio o carácter extraño \xa0) es igual a "ND".

#.shape te dice cuántas filas tienen ese valor "ND".
#Esto sirve para ver cuántas entradas “no detectadas” hay.

df[df["Zinc disponible doble \xa0acido"] == "ND"].shape



In [ ]:
df[df["Zinc disponible doble \xa0acido"] == "ND"]


In [ ]:
break

In [ ]:
#Imprime la lista de columnas como una lista de Python.
print(df.columns.tolist())

['Secuencial', 'Fecha de Análisis', 'Departamento', 'Municipio', 'Cultivo', 'Estado', 'Tiempo de establecimiento', 'Topografia', 'Drenaje', 'Riego', 'Fertilizantes aplicados', 'pH agua:suelo', 'Materia organica', 'Fósforo Bray II', 'Azufre Fosfato monocalcico', 'Acidez Intercambiable', 'Aluminio intercambiable', 'Calcio intercambiable', 'Magnesio intercambiable', 'Potasio intercambiable', 'Sodio intercambiable', 'capacidad de intercambio cationico', 'Conductividad electrica', 'Hierro disponible olsen', 'Cobre disponible', 'Manganeso disponible Olsen', 'Zinc disponible Olsen', 'Boro disponible', 'Hierro disponible doble acido', 'Cobre disponible doble acido', 'Manganeso disponible doble acido', 'Zinc disponible doble \xa0acido']


# Definir qué columnas son numéricas (propiedades químicas)

In [ ]:
# Todas las columnas químicas numéricas (desde pH hasta el último metal)

#Defino una lista llamada numeric_cols que incluye todas las columnas
#que necesito tratar como variables numéricas químicas.
#Esta lista es la base para aplicar los procesos de limpieza, detección de outliers
#y demás transformaciones, asegurando que estos procedimientos se ejecuten únicamente
#sobre las columnas que efectivamente contienen información numérica.

numeric_cols = [
    'pH agua:suelo',
    'Materia organica',
    'Fósforo Bray II',
    'Azufre Fosfato monocalcico',
    'Acidez Intercambiable',
    'Aluminio intercambiable',
    'Calcio intercambiable',
    'Magnesio intercambiable',
    'Potasio intercambiable',
    'Sodio intercambiable',
    'capacidad de intercambio cationico',
    'Conductividad electrica',
    'Hierro disponible olsen',
    'Cobre disponible',
    'Manganeso disponible Olsen',
    'Zinc disponible Olsen',
    'Boro disponible',
    'Hierro disponible doble acido',
    'Cobre disponible doble acido',
    'Manganeso disponible doble acido',
    'Zinc disponible doble \xa0acido',
]


# Función de limpieza de valores

Objetivo:

Quitar espacios.

Convertir comas decimales a punto.

Tratar expresiones tipo "<0.09" → usamos la mitad del límite (0.09/2).

Dejar como NaN las cadenas vacías o códigos raros.

In [ ]:
#Esta función limpia textos, símbolos, límites de detección y errores del CSV, y devuelve siempre un número usable o NaN


def limpiar_valor(x):
    """
    Limpia un valor individual proveniente del CSV
    y lo convierte en float o NaN.
    """
    if pd.isna(x):                 # Si ya es NaN -> lo devolvemos igual
        return np.nan

    x = str(x).strip()             # Pasar a string y quitar espacios

    # Valores que equivalen a dato faltante
    if x == '' or x.lower() in ['na', 'nd', 'nr', 's/r', 'nan']:
        return np.nan

    # Valores tipo "<0.09" (por debajo del límite de detección)
    if x.startswith('<'):
        try:
            limite = float(x[1:].replace(',', '.'))  # tomar la parte numérica
            return limite / 2                        # decisión: usar la mitad del límite
        except:
            return np.nan                            # si no podemos parsear, lo dejamos como NaN

    # Reemplazar coma decimal por punto
    x = x.replace(',', '.')

    # Intentar convertir a float
    try:
        return float(x)
    except:
        return np.nan


In [ ]:
#Conversión a numérico y limpieza básica

#Recorre cada columna numérica.
#Intenta convertirla a número (float).
#Si encuentra algo no convertible (por ejemplo "ND"), lo convierte en NaN (errors="coerce").
#Esto homogeneiza los tipos de datos para poder hacer estadística.

import numpy as np

# Aplicar la limpieza a cada columna numérica
for col in numeric_cols:
    df[col] = df[col].apply(limpiar_valor)

# Opcional: ver un resumen estadístico de estas columnas ya como numéricas
print(df[numeric_cols].describe().T)

                                      count        mean         std    min  \
pH agua:suelo                       92733.0    5.707986    1.020383  1.900   
Materia organica                    92734.0    4.408162    4.592723  0.000   
Fósforo Bray II                     91269.0   28.704551   74.550168  0.088   
Azufre Fosfato monocalcico          91239.0   14.255018   56.069428 -1.010   
Acidez Intercambiable               47608.0    2.300043    2.346674  0.000   
Aluminio intercambiable             47608.0    1.854159    2.029772  0.000   
Calcio intercambiable               92738.0    7.386947    9.411489  0.028   
Magnesio intercambiable             92738.0    2.039970    2.744472  0.006   
Potasio intercambiable              92738.0    0.367531    0.706021  0.045   
Sodio intercambiable                92738.0    0.218302    0.945157 -0.040   
capacidad de intercambio cationico  92671.0   11.253584   10.923775  0.130   
Conductividad electrica             91710.0    0.430402    1.058

In [ ]:
# Función para detectar outliers con IQR

#Esta función implementa la detección de outliers por método IQR:
#Calcula Q1, Q3 e IQR = Q3 − Q1.
#Define límites: lim_inf y lim_sup.
#Crea una máscara booleana (True = outlier).

#Devuelve:

#mask: serie booleana,

#(lim_inf, lim_sup): los límites numéricos.

def detectar_outliers_iqr(serie):
    """
    Recibe una serie numérica (columna) y devuelve:
    - máscara booleana con True en los outliers
    - tupla con (lim_inf, lim_sup)
    """
    q1 = serie.quantile(0.25)              # Primer cuartil
    q3 = serie.quantile(0.75)              # Tercer cuartil
    iqr = q3 - q1                          # Rango intercuartílico
    lim_inf = q1 - 1.5 * iqr               # Límite inferior
    lim_sup = q3 + 1.5 * iqr               # Límite superior
    mask = (serie < lim_inf) | (serie > lim_sup)  # True donde hay outlier
    return mask, (lim_inf, lim_sup)


In [ ]:
#Creo un DataFrame llamado outlier_df, paralelo a df, que indica para cada fila y
#para cada columna numérica si el valor corresponde a un outlier (True) o no (False).

#Para cada columna numérica:

#Aplico la función detectar_outliers_iqr únicamente sobre los valores no nulos.
#Reindexo la máscara resultante para que coincida con todas las filas del DataFrame original.
#Al final, guardo esa máscara booleana en outlier_df bajo el nombre de la columna correspondiente.


# Inicializamos un dataframe de banderas de outliers con False
outlier_df = pd.DataFrame(False, index=df.index, columns=numeric_cols)

# Recorremos cada columna numérica
for col in numeric_cols:
    serie = df[col]                                  # tomamos la columna
    mask, limites = detectar_outliers_iqr(serie.dropna())  # aplicamos IQR sobre valores no nulos
    mask = mask.reindex(df.index, fill_value=False) # reindexamos para cubrir todas las filas
    outlier_df[col] = mask                          # guardamos la máscara en outlier_df


# Reglas de rango físico

pH debe estar entre 0 y 14.

Para el resto de variables: valores negativos se consideran físicamente imposibles.

In [ ]:
# Inicializamos otro dataframe de banderas para errores físicos
fisico_df = pd.DataFrame(False, index=df.index, columns=numeric_cols)

# Regla para pH: debe estar entre 0 y 14
fisico_df['pH agua:suelo'] = (
    (df['pH agua:suelo'] < 0) |
    (df['pH agua:suelo'] > 14)
)

# Regla para el resto: no se permiten valores negativos
for col in numeric_cols:
    if col != 'pH agua:suelo':
        fisico_df[col] = df[col] < 0

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


# Datos faltantes (NaN)

In [ ]:
# Banderas de datos faltantes por columna
missing_df = df[numeric_cols].isna()
missing_df

,pH agua:suelo,Materia organica,Fósforo Bray II,Azufre Fosfato monocalcico,Acidez Intercambiable,Aluminio intercambiable,Calcio intercambiable,Magnesio intercambiable,Potasio intercambiable,Sodio intercambiable,...,Conductividad electrica,Hierro disponible olsen,Cobre disponible,Manganeso disponible Olsen,Zinc disponible Olsen,Boro disponible,Hierro disponible doble acido,Cobre disponible doble acido,Manganeso disponible doble acido,Zinc disponible doble acido
0,False,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92733,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
92734,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
92735,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
92736,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True


# Construcción del Índice de Calidad de Datos (ICD)

Definamos que para cada celda (fila–columna) hay un error si:

el valor es faltante (missing), o

es outlier por IQR, o

viola una regla física.

Entonces el ICD por fila será:

ICD\_fila = 1 - \frac{\text{# celdas con error en esa fila}}{\text{# columnas numéricas}}

Es decir, 1 = fila perfecta, 0 = todos los datos problemáticos.

In [ ]:
# Unificamos las banderas: hay error si cualquiera de las tres condiciones se cumple
errores_df = missing_df | outlier_df | fisico_df

# Número total de variables numéricas consideradas
n_vars = len(numeric_cols)

# Cálculo del ICD por fila (registro)
df['ICD_fila'] = 1 - errores_df.sum(axis=1) / n_vars

# ICD global del dataset (promedio de todas las filas)
ICD_global = df['ICD_fila'].mean()

print("Índice de Calidad de Datos (ICD) global:", ICD_global)


Índice de Calidad de Datos (ICD) global: 0.6979447475684185


# Resumen de calidad por columna

Esto te sirve para el informe del reto: cuántos missing, outliers y errores físicos tiene cada variable.

In [ ]:
# Conteo de errores por columna
resumen_columnas = pd.DataFrame({
    'missing': missing_df.sum(),          # cuántos NaN por columna
    'outliers_IQR': outlier_df.sum(),     # cuántos outliers por columna
    'errores_fisicos': fisico_df.sum()    # cuántos fuera de rango físico por columna
})

# Agregar total de errores y porcentaje sobre el total de filas
resumen_columnas['total_errores'] = resumen_columnas.sum(axis=1)
resumen_columnas['porc_filas_con_error_%'] = 100 * resumen_columnas['total_errores'] / len(df)

print(resumen_columnas.sort_values('total_errores', ascending=False))


                                    missing  outliers_IQR  errores_fisicos  \
Zinc disponible doble  acido          87952           550                0   
Manganeso disponible doble acido      87952           418                0   
Hierro disponible doble acido         87952           285                0   
Cobre disponible doble acido          87952           132                0   
Acidez Intercambiable                 45130          2735                0   
Aluminio intercambiable               45130          2677                0   
Zinc disponible Olsen                  5872          8630                0   
Manganeso disponible Olsen             5872          6975                0   
Hierro disponible olsen                5872          6462                0   
Fósforo Bray II                        1469          9706                0   
Sodio intercambiable                      0         10912                1   
Azufre Fosfato monocalcico             1499          8858       